In [ ]:
from owslib.wmts import WebMapTileService
from IPython.display import Image, display
import time
import math
import WMTS_General_Map as general
config = general.get_config_from_json()

## Experimental Testing for getting photo's out of multiple WMTS servers.
This Notebook experiments with a way  to get photo's as a datasource to later integrate it into the SailingRobots Website. 

The static variables like the website and the normal map layer.

In [ ]:
try:
    wmts_avoin = WebMapTileService(config["wmts"]["avoin"])
    time.sleep(0.01)
    wmts_maasto = WebMapTileService(config["wmts"]["maasto"])
    time.sleep(0.01)
    wmts_ava = WebMapTileService(config["wmts"]["ava"])
    
except Exception as e:
    print(e)
tile_matrix_set_name = config["matrix"]["set_name"]
map_layer = config["matrix"]["map_layer"]["orto"]
standardized_rendering_pixel_size = config["standardized_rendering_pixel_size"]

Get a single image information calculated by the coordinates given. This is so that we can later get images for multiple points and see which of these is at a certain water depth.

In [ ]:
def get_single_height_width(matrix, lat, lon,convert):
    delta_width = matrix.tilewidth * standardized_rendering_pixel_size * matrix.scaledenominator
    delta_height = matrix.tileheight * standardized_rendering_pixel_size * matrix.scaledenominator
    if convert:
        finn_coordinates = (lat,lon)
    else:
        finn_coordinates = general.convert_coordinate_systems(lat=lat, lon=lon, inverse=True)
    

    width = math.trunc((finn_coordinates[0] - matrix.topleftcorner[0]) / delta_width)
    height = math.trunc((matrix.topleftcorner[1] - finn_coordinates[1]) / delta_height)
    return height, width

If the tileset is not given it calculates the matrix and and set's the default level.

In [ ]:
def get_column_row_for_coordinate_at_level_single(latitude, longitude, wmts,tile_matrix_set_name_,convert, tileset=None, level=2):
    if tileset == None:
        tileset = wmts.tilematrixsets[tile_matrix_set_name_]
        list_of_tilematrixes = list(tileset.tilematrix.keys())
    matrix = tileset.tilematrix[list_of_tilematrixes[level]]
    codes = get_single_height_width(
        matrix, 
        lat=latitude, 
        lon=longitude,
        convert=convert)
    if codes[1] > matrix.matrixwidth or codes[1] > matrix.matrixwidth:
        print('hello',codes)
    return codes

This method executes the other methods and displays the image. 

In [ ]:
def get_image_for_coordinate(latitude, longitude, level, wmts_,tile_matrix_set_name_, specified_map_layer=map_layer,special_level=False,convert=False,split_up=False):
    tile_level = '{0}'.format(level)
    if special_level:
        tile_level = '{0}:{1}'.format(tile_matrix_set_name_,level)
        tms = wmts_test.contents[specified_map_layer]
        while tile_level not in tms.tilematrixsetlinks[tile_matrix_set_name_].tilematrixlimits:
            level -=1
            tile_level = '{0}:{1}'.format(tile_matrix_set_name_,level)
    codes = get_column_row_for_coordinate_at_level_single(latitude=latitude,
                                                          longitude=longitude,
                                                          wmts=wmts_,
                                                          level=level,
                                                         tile_matrix_set_name_=tile_matrix_set_name_,convert=convert)
    row, column = codes
    found=True
    if split_up:
        found = False
        i=-1
    while not found:
        i+=1
        tms = wmts_test.contents[specified_map_layer+'{0}'.format(i)]
        limits = tms.tilematrixsetlinks[tile_matrix_set_name_].tilematrixlimits[tile_level]
        if (limits.mintilerow <= row <= limits.maxtilerow and limits.mintilecol <= column <= limits.maxtilecol):
            found = True
        if i<10:
            break;
        
    tile = wmts_.gettile(layer=specified_map_layer,
                         tilematrixset=tile_matrix_set_name_,
                         tilematrix='{0}'.format(tile_level),
                         row=codes[0],
                         column=codes[1],
                         format = wmts_.contents[specified_map_layer].formats[-1])
    return Image(tile.read())

The execution of getting a certain images from the coordinates file. it also has an infrared option at maasto

In [ ]:
info_dict = general.get_coordinates_from_file()[0]
img = get_image_for_coordinate(info_dict['latitude'], 
                         info_dict['longitude'], 
                         info_dict['level'], 
                         wmts_=wmts_avoin,
                              tile_matrix_set_name_=tile_matrix_set_name)
img_2 = get_image_for_coordinate(info_dict['latitude'], 
                         info_dict['longitude'], 
                         info_dict['level'], 
                         wmts_=wmts_maasto,
                                tile_matrix_set_name_=tile_matrix_set_name)
img_3 = get_image_for_coordinate(info_dict['latitude'], 
                         info_dict['longitude'], 
                         info_dict['level'], 
                         wmts_=wmts_maasto, 
                         specified_map_layer=config["matrix"]["map_layer"]["infrared"],
                                tile_matrix_set_name_=tile_matrix_set_name)
img_4 = get_image_for_coordinate(info_dict['latitude'], 
                         info_dict['longitude'], 
                         info_dict['level'], 
                         wmts_=wmts_ava, 
                         specified_map_layer=config['matrix']['map_layer']['ava_infrared'],
                            tile_matrix_set_name_=config['coordinates_system']['ETRS-TM35FIN'],
                                 special_level=True)
img_5 = get_image_for_coordinate(info_dict['latitude'], 
                         info_dict['longitude'], 
                         info_dict['level'], 
                         wmts_=wmts_ava, 
                         specified_map_layer=config['matrix']['map_layer']['ava_norm'],
                            tile_matrix_set_name_=config['coordinates_system']['ETRS-TM35FIN'],
                                 special_level=True)
img_6 = get_image_for_coordinate(info_dict['latitude'], 
                         info_dict['longitude'], 
                         info_dict['level'], 
                         wmts_=wmts_ava, 
                         specified_map_layer=config['matrix']['map_layer']['ava_norm'],
                            tile_matrix_set_name_=config['coordinates_system']['ETRS-TM35FIN'],
                                 special_level=True,split_up=True)
img_7= get_image_for_coordinate(info_dict['latitude'], 
                         info_dict['longitude'], 
                         info_dict['level'], 
                         wmts_=wmts_ava, 
                         specified_map_layer=config['matrix']['map_layer']['ava_normal_color'],
                            tile_matrix_set_name_=config['coordinates_system']['ETRS-TM35FIN'],
                                 special_level=True)

display(img,img_2,img_7)
display(img_4)
display(img_5)
display(img_6)